# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
def sampling(sample):
    lat = sample.get('latitude')
    lon = sample.get('longitude')
    ch4 = sample.get('ch4')
    return ee.Feature(ee.Geometry.Point([lon, lat]), {'ch4': ch4})

# Import two weeks of S5P methane and composite by mean.
ch4 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4') \
  .select('CH4_column_volume_mixing_ratio_dry_air') \
  .filterDate('2019-08-01', '2019-08-15') \
  .mean() \
  .rename('ch4')

# Define an area to perform interpolation over.
aoi = ee.Geometry.Polygon(
    [[[-95.68487605978851, 43.09844605027055],
       [-95.68487605978851, 37.39358590079781],
       [-87.96148738791351, 37.39358590079781],
       [-87.96148738791351, 43.09844605027055]]], {}, False)

# Sample the methane composite to generate a FeatureCollection.
samples = ch4.addBands(ee.Image.pixelLonLat()) \
  .sample(**{'region': aoi, 'numPixels': 1500,
    'scale':1000, 'projection': 'EPSG:4326'}) \
  .map(sampling)

# Combine mean and standard deviation reducers for efficiency.
combinedReducer = ee.Reducer.mean().combine(**{
  'reducer2': ee.Reducer.stdDev(),
  'sharedInputs': True})

# Estimate global mean and standard deviation from the points.
stats = samples.reduceColumns(**{
  'reducer': combinedReducer,
  'selectors': ['ch4']})

# Do the interpolation, valid to 70 kilometers.
interpolated = samples.inverseDistance(**{
  'range': 7e4,
  'propertyName': 'ch4',
  'mean': stats.get('mean'),
  'stdDev': stats.get('stdDev'),
  'gamma': 0.3})

# Define visualization arguments.
band_viz = {
  'min': 1800,
  'max': 1900,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678',
            'EB7852', 'FBB32F', 'F0F921']}

# Display to map.
# Map.centerObject(ee.FeatureCollection(aoi), 7)
ee_layers.append(EarthEngineLayer(ee_object=ch4, vis_params=band_viz))
# Map.addLayer(interpolated, band_viz, 'CH4 Interpolated')




Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…